Pair Programming ETL IV Carga I

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras. Una de las tablas contendrá la información que obtuvimos de los ejercicios de pair programming de Limpieza, es decir, el data set de ataques de tiburones limpito. La segunda tabla tendrá la información obtenida en el ejercicio de pair de ETL 1.
📌 Nota Todo lo tendremos que hacer desde jupyter notebook
  1. Cread la BBDD con el nombre de tiburones.
  2. Cread las tablas de la BBDD:
        - Tabla ataques
        - Tabla clima
  3. BONUS Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación
🚨 En caso de que no tengáis los datos unidos de la sesión anterior, tenéis un csv datos_union_clima_ataques con todos los datos que necesitareis para este ejercicio

In [31]:
import pandas as pd
import numpy as np
import requests
import mysql.connector

In [32]:
df = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/datos_union_clima_ataques.csv',index_col = 0)
df.head(2)

,case_number,year,type,country,area,location,activity,name,age,time,...,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,14h00 -15h00,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


- 1-CREACIÓN BBDD

In [48]:
def crear_bbdd(nombre_bbdd):
    
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="Marinadalab" # aquí tendréis que poner vuestra contraseña de MySQL
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [34]:
mydb = crear_bbdd('tiburones')

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


- 2-CREACIÓN TABLAS

In [49]:
# primero crearnos una función que valga para crear o insertar datos de cq tabla, con 3 parametros: 
# Nombre BBDD           # Contraseña servidor de SQL    # Query a insertar
                                                                                            

def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [36]:
df.columns

Index(['case_number', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'age', 'time', 'species_', 'injury', 'fecha_limpia',
       'fatal', 'sex', 'pais', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb',
       'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb',
       'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb',
       'wind_950mb', 'wind_900mb', 'wind_850mb', 'wind_800mb', 'wind_750mb',
       'wind_700mb', 'wind_650mb', 'wind_600mb', 'wind_550mb', 'wind_500mb',
       'wind_450mb', 'wind_400mb', 'wind_350mb', 'wind_300mb', 'wind_250mb',
       'wind_200mb', 'index', 'timepoint', 'cloudcover', 'highcloud',
       'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m',
       'msl_pressure', 'prec_amount', 'snow_depth', 'wind10m.direction',
       'wind10m.speed'],
      dtype='object')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4395 entries, 0 to 4394
Data columns (total 62 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   case_number        4394 non-null   object 
 1   year               4395 non-null   float64
 2   type               4394 non-null   object 
 3   country            4395 non-null   object 
 4   area               4363 non-null   object 
 5   location           4276 non-null   object 
 6   activity           4066 non-null   object 
 7   name               4325 non-null   object 
 8   age                2681 non-null   object 
 9   time               2395 non-null   object 
 10  species_           2647 non-null   object 
 11  injury             4381 non-null   object 
 12  fecha_limpia       4148 non-null   object 
 13  fatal              3975 non-null   object 
 14  sex                4059 non-null   object 
 15  pais               4395 non-null   object 
 16  rh_950mb           4395 

In [50]:
# Definimos las distintas *queries* que crearán nuestras tablas. 
# Tabla ataques

tabla_ataques = '''
CREATE TABLE IF NOT EXISTS 'tiburones'.'ataques' (
  'idataque' INT NOT NULL AUTO_INCREMENT,
  'year' DECIMAL NOT NULL,
  'type' VARCHAR (100) NOT NULL,
  'country' VARCHAR (100) NOT NULL,
  'area' VARCHAR (100) NOT NULL,
  'location' VARCHAR (100) NOT NULL,
  'activity' VARCHAR (100) NOT NULL,
  'name' VARCHAR (100) NOT NULL,
  'age' VARCHAR (100) NOT NULL,
  'time' VARCHAR (100) NOT NULL,
  'species_' VARCHAR (100) NOT NULL,
  'injury'  VARCHAR (100) NOT NULL,
  'fecha_limpia' VARCHAR (100) NOT NULL,
  'fatal' VARCHAR (100) NOT NULL,
  'sex' VARCHAR (100) NOT NULL,
  'país' VARCHAR (100) NOT NULL,
  PRIMARY KEY ('idataque'))
ENGINE = InnoDB;
'''




In [51]:
# Tabla clima

tabla_clima = """
CREATE TABLE IF NOT EXISTS 'tiburones.'clima' (
  'idclima' INT NOT NULL,
  'rh_950mb' DECIMAL NOT NULL,        
  'rh_900mb' DECIMAL NOT NULL,          
  'rh_850mb' DECIMAL NOT NULL,          
  'rh_800mb' DECIMAL NOT NULL,         
  'rh_750mb' DECIMAL NOT NULL,        
  'rh_700mb 'DECIMAL NOT NULL,         
  'rh_650mb' DECIMAL NOT NULL,         
  'rh_600mb' DECIMAL NOT NULL,          
  'rh_550mb' DECIMAL NOT NULL,       
  'rh_500mb' DECIMAL NOT NULL,         
  'rh_450mb' DECIMAL NOT NULL,          
  'rh_400mb' DECIMAL NOT NULL,          
  'rh_350mb' DECIMAL NOT NULL,         
  'rh_300mb' DECIMAL NOT NULL,          
  'rh_250mb' DECIMAL NOT NULL,          
  'rh_200mb' DECIMAL NOT NULL,          
  'wind_950mb' DECIMAL NOT NULL,       
  'wind_900mb' DECIMAL NOT NULL,       
  'wind_850mb' DECIMAL NOT NULL,        
  'wind_800mb' DECIMAL NOT NULL,        
  'wind_750mb' DECIMAL NOT NULL,        
  'wind_700mb' DECIMAL NOT NULL,       
  'wind_650mb' DECIMAL NOT NULL,        
  'wind_600mb' DECIMAL NOT NULL,        
  'wind_550mb' DECIMAL NOT NULL,       
  'wind_500mb' DECIMAL NOT NULL,       
  'wind_450mb' DECIMAL NOT NULL,       
  'wind_400mb' DECIMAL NOT NULL,       
  'wind_350mb' DECIMAL NOT NULL,       
  'wind_300mb' DECIMAL NOT NULL,       
  'wind_250mb' DECIMAL NOT NULL,       
  'wind_200mb' DECIMAL NOT NULL,       
  'index' DECIMAL NOT NULL,        
  'timepoint' DECIMAL NOT NULL,         
  'cloudcover' DECIMAL NOT NULL,         
  'highcloud' DECIMAL NOT NULL,          
  'midcloud' DECIMAL NOT NULL,         
  'lowcloud' DECIMAL NOT NULL,          
  'temp2m' DECIMAL NOT NULL,            
  'lifted_index' DECIMAL NOT NULL,      
  'rh2m' DECIMAL NOT NULL,               
  'msl_pressure' DECIMAL NOT NULL,       
  'prec_amount' DECIMAL NOT NULL,       
  'snow_depth'DECIMAL NOT NULL,        
  'wind10m.direction'DECIMAL NOT NULL, 
  'wind10m.speed' DECIMAL NOT NULL,
  PRIMARY KEY ('idclima'))
ENGINE = InnoDB;"""
      

In [53]:
# Llamamos a la función para crear cada una de las tablas de nuestra BBDD.

crear_insertar_tabla('tiburones', 'Marinadalab', tabla_ataques)  


1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ''tiburones'.'ataques' (
  'idataque' INT NOT NULL AUTO_INCREMENT,
  'year' DECIM' at line 1
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ''tiburones'.'ataques' (
  'idataque' INT NOT NULL AUTO_INCREMENT,
  'year' DECIM' at line 1


In [54]:
crear_insertar_tabla('tiburones', 'Marinadalab', tabla_clima)  


1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ''tiburones.'clima' (
  'idclima' INT NOT NULL,
  'rh_950mb' DECIMAL NOT NULL,   ' at line 1
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ''tiburones.'clima' (
  'idclima' INT NOT NULL,
  'rh_950mb' DECIMAL NOT NULL,   ' at line 1


- 3-INSERCIÓN DATOS

In [55]:

for indice, fila in df.iterrows(): 
        query_ataques = f"""
            INSERT INTO ataques (year, type, country, area, location, activity, name, age,time,
            species_ , injury , fecha_limpia , fatal , sex, país 
            VALUES ("{fila['year']}", "{fila['type']}","{fila['country']}","{fila['area']}","{fila['location']}",
            "{fila['activity']}","{fila['name']}","{fila['age']}","{fila['time']}","{fila['species_']}","{fila['injury']}",
            "{fila['fecha_limpia']}","{fila['fatal']}","{fila['sex']}","{fila['pais']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
crear_insertar_tabla("tiburones", "Marinadalab", query_ataques)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'VALUES ("0.0", "Sea Disaster","PAPUA NEW GUINEA","Madang Province","Off Lae",
  ' at line 3
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'VALUES ("0.0", "Sea Disaster","PAPUA NEW GUINEA","Madang Province","Off Lae",
  ' at line 3


In [57]:
for indice, fila in df.iterrows(): 
        query_clima = f"""
            INSERT INTO ataques ( rh_950mb, rh_900mb, rh_850mb, rh_800mb,
            rh_750mb, rh_700mb, rh_650mb, rh_600mb, rh_550mb, rh_500mb,
            rh_450mb, rh_400mb, rh_350mb, rh_300mb, rh_250mb, rh_200mb,
            wind_950mb, wind_900mb, wind_850mb, wind_800mb, wind_750mb,
            wind_700mb, wind_650mb, wind_600mb, wind_550mb, wind_500mb,
            wind_450mb, wind_400mb, wind_350mb, wind_300mb, wind_250mb,
            wind_200mb, index, timepoint, cloudcover, highcloud,
            midcloud, lowcloud, temp2m, lifted_index, rh2m,
            msl_pressure, prec_amount, snow_depth, wind10m.direction, wind10m.speed
                    
            VALUES ( "{fila["rh_950mb"]}", "{fila['rh_900mb']},"{fila['rh_850mb']},"{fila['rh_800mb']}
            "{fila['rh_750mb']},"{fila['rh_700mb']},"{fila['rh_650mb']},"{fila['rh_600mb']},
            "{fila['rh_550mb']},"{fila['rh_500mb']},"{fila['rh_450mb']},"{fila['rh_400mb']},
            "{fila['rh_350mb']},"{fila['rh_300mb']}","{fila['rh_250mb']}","{fila['rh_200mb']}")
            "{fila['wind_950mb']},"{fila['rh_900mb']}","{fila['rh_850mb']},"{fila['rh_800mb']},
            "{fila['rh_750mb']},"{fila['rh_700mb']},"{fila['rh_650mb']},"{fila['rh_600mb']},
            "{fila['rh_550mb']},"{fila['rh_500mb']},"{fila['rh_350mb']},"{fila['rh_350mb']},
            "{fila['rh_300mb']},"{fila['rh_250mb']},"{fila['rh_200mb']},"{fila['index']},
            "{fila['timepoint']}","{fila['cloudcover']}","{fila['highcloud']}","{fila['cloudcover']}",
            "{fila['midcloud']}","{fila['lowcloud']}","{fila['temp2m']}","{fila['lifted_index']}",
            "{fila['rh2m']}","{fila['msl_pressure']}","{fila['prec_amount']}","{fila['snow_depth']}",
            "{fila['wind10m.direction']}","{fila['wind10m.speed']}";
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
crear_insertar_tabla("tiburones", "Marinadalab", query_clima)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'index, timepoint, cloudcover, highcloud,
            midcloud, lowcloud, temp2m,' at line 7
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'index, timepoint, cloudcover, highcloud,
            midcloud, lowcloud, temp2m,' at line 7
